In [48]:
import pandas as pd
import numpy as np
import sklearn

In [49]:
print(f'pandas=={pd.__version__}')
print(f'numpy=={np.__version__}')
print(f'sklearn=={sklearn.__version__}')


pandas==2.3.1
numpy==2.3.1
sklearn==1.7.0


In [50]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [51]:
data_url = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

df = pd.read_csv(data_url)

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [52]:
y_train = df.churn

In [53]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [54]:
dv = DictVectorizer()

train_dict = df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'liblinear'
,max_iter,100
,multi_class,'deprecated'


#### make a prediction

In [55]:
costumer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

X = dv.transform(costumer)

churn = model.predict_proba(X)[0, 1]
churn

np.float64(0.6638197301805238)

#### Save the model

In [56]:
import pickle

In [45]:
#output_file = 'model.bin'

#f_out = open(output_file, 'wb')
#pickle.dump((dv, model), f_out)
#f_out.close()
# instead of doing this because we might forget to close the file we can use with
# wb is for writing binary file

In [57]:
with open('model.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

# automatically closes after the with statement


In [42]:
#!ls -lh

total 84K
-rw-rw-rw- 1 codespace codespace  78K Nov  7 16:42 churn_prediction.ipynb
-rw-rw-rw- 1 codespace codespace 2.5K Nov  7 16:43 model.bin


#### load the model

In [49]:
#model_file = 'model.bin'

In [58]:
with open('model.bin', 'rb') as f_in:
    (dv, model) = pickle.load(f_in)

In [59]:
dv, model

(DictVectorizer(), LogisticRegression(solver='liblinear'))

In [60]:
costumer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

X = dv.transform(costumer)

churn = model.predict_proba(X)[0, 1]

if churn >= 0.5:
    print('send email with promo')
else:
    print('dont do anything')

send email with promo


In [53]:
churn

np.float64(0.6273177838726128)

#### Make a pipeline

In [61]:
from sklearn.pipeline import make_pipeline

In [62]:
pipeline = make_pipeline(
    DictVectorizer(),
    LogisticRegression(solver='liblinear')
)

In [64]:
train_dict = df[categorical + numerical].to_dict(orient='records')

pipeline.fit(train_dict, y_train)

,steps,"[('dictvectorizer', ...), ('logisticregression', ...)]"
,transform_input,None
,memory,None
,verbose,False
,dtype,<class 'numpy.float64'>
,separator,'='
,sparse,True
,sort,True
,penalty,'l2'
,dual,False
,tol,0.0001


save the model

In [66]:
with open('model.bin', 'wb') as f_out:
    pickle.dump(pipeline, f_out)

In [67]:
with open('model.bin', 'rb') as f_in:
    pipeline = pickle.load(f_in)

In [68]:
costumer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}


churn = pipeline.predict_proba(costumer)[0, 1]
print('prob of churn =',churn)

if churn >= 0.5:
    print('send email with promo')
else:
    print('dont do anything')

prob of churn = 0.6638197301805238
send email with promo
